In [ ]:
import ray

# Load data.
dataset = ray.data.read_csv("s3://anonymous@air-example-data/breast_cancer.csv")

# Split data into train and validation.
train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)

# Create a test dataset by dropping the target column.
test_dataset = valid_dataset.drop_columns(cols=["target"])

2023-05-19 22:20:56,826	ERROR services.py:1197 -- Failed to start the dashboard , return code -11
2023-05-19 22:20:56,828	ERROR services.py:1222 -- Error should be written to 'dashboard.log' or 'dashboard.err'. We are printing the last 20 lines for you. See 'https://docs.ray.io/en/master/ray-observability/ray-logging.html#logging-directory-structure' to find where the log file is.
2023-05-19 22:20:56,829	ERROR services.py:1266 -- 
The last 20 lines of /tmp/ray/session_2023-05-19_22-20-56_014030_2633550/logs/dashboard.log (it contains the error message from the dashboard): 
2023-05-19 22:20:56,655	INFO head.py:239 -- Starting dashboard metrics server on port 44227

2023-05-19 22:20:56,958	INFO worker.py:1625 -- Started a local Ray instance.


In [2]:
import ray

In [ ]:
ray.init()

2023-05-19 23:20:28,482	ERROR services.py:1197 -- Failed to start the dashboard , return code -11
2023-05-19 23:20:28,484	ERROR services.py:1222 -- Error should be written to 'dashboard.log' or 'dashboard.err'. We are printing the last 20 lines for you. See 'https://docs.ray.io/en/master/ray-observability/ray-logging.html#logging-directory-structure' to find where the log file is.
2023-05-19 23:20:28,484	ERROR services.py:1266 -- 
The last 20 lines of /tmp/ray/session_2023-05-19_23-20-26_717211_2634598/logs/dashboard.log (it contains the error message from the dashboard): 
2023-05-19 23:20:28,358	INFO head.py:239 -- Starting dashboard metrics server on port 44227

2023-05-19 23:20:28,619	INFO worker.py:1625 -- Started a local Ray instance.


In [ ]:
import ray
import time


# A regular Python function.
def normal_function():
    return 1


# By adding the `@ray.remote` decorator, a regular Python function
# becomes a Ray remote function.
@ray.remote
def my_function():
    return 1


# To invoke this remote function, use the `remote` method.
# This will immediately return an object ref (a future) and then create
# a task that will be executed on a worker process.
obj_ref = my_function.remote()

# The result can be retrieved with ``ray.get``.
assert ray.get(obj_ref) == 1


@ray.remote
def slow_function():
    time.sleep(10)
    return 1


# Ray tasks are executed in parallel.
# All computation is performed in the background, driven by Ray's internal event loop.
for _ in range(4):
    # This doesn't block.
    slow_function.remote()

2023-05-20 15:37:57,933	ERROR services.py:1197 -- Failed to start the dashboard , return code -11
2023-05-20 15:37:57,935	ERROR services.py:1222 -- Error should be written to 'dashboard.log' or 'dashboard.err'. We are printing the last 20 lines for you. See 'https://docs.ray.io/en/master/ray-observability/ray-logging.html#logging-directory-structure' to find where the log file is.
2023-05-20 15:37:57,936	ERROR services.py:1266 -- 
The last 20 lines of /tmp/ray/session_2023-05-20_15-37-56_008505_2652268/logs/dashboard.log (it contains the error message from the dashboard): 
2023-05-20 15:37:57,731	INFO head.py:239 -- Starting dashboard metrics server on port 44227

2023-05-20 15:37:58,062	INFO worker.py:1625 -- Started a local Ray instance.


In [4]:
import torch
import torch.nn as nn

import ray
from ray import train
from ray.air import session, Checkpoint
from ray.train.torch import TorchTrainer
from ray.air.config import ScalingConfig


# If using GPUs, set this to True.
num_workers=1
use_gpu = True


input_size = 1
layer_size = 15
output_size = 1
num_epochs = 3


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, layer_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(layer_size, output_size)

    def forward(self, input):
        return self.layer2(self.relu(self.layer1(input)))


def train_loop_per_worker():
    dataset_shard = session.get_dataset_shard("train")
    model = NeuralNetwork()
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

    model = train.torch.prepare_model(model)

    for epoch in range(num_epochs):
        for batches in dataset_shard.iter_torch_batches(
            batch_size=32, dtypes=torch.float
        ):
            inputs, labels = torch.unsqueeze(batches["x"], 1), batches["y"]
            output = model(inputs)
            loss = loss_fn(output, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print(f"epoch: {epoch}, loss: {loss.item()}")

        session.report(
            {},
            checkpoint=Checkpoint.from_dict(
                dict(epoch=epoch, model=model.state_dict())
            ),
        )


train_dataset = ray.data.from_items([{"x": x, "y": 2 * x + 1} for x in range(200)])
scaling_config = ScalingConfig(num_workers=num_workers, use_gpu=use_gpu)
trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    scaling_config=scaling_config,
    datasets={"train": train_dataset},
)
result = trainer.fit()

(raylet) [2023-05-31 22:12:04,701 E 1453491 1453600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-05-31_22-08-32_654304_1453193 is over 95% full, available space: 17757290496; capacity: 418112987136. Object creation will fail if spilling is required.
(RayTrainWorker pid=1456225) 2023-05-31 22:12:08,077	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=1455144) /home/modongsong/.pyenv/versions/miniconda3-4.7.12/envs/jupyterlab/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size. [repeated 29x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RayTra

(pid=1456225) - RandomizeBlockOrder 1:   0%|          | 0/200 [00:00<?, ?it/s]

(pid=1456225) Running 0:   0%|          | 0/200 [00:00<?, ?it/s]

(RayTrainWorker pid=1455141) epoch: 2, loss: 246481479008256.0 [repeated 29x across cluster]


(RayTrainWorker pid=1456225) 2023-05-31 22:12:09,486	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[RandomizeBlockOrder]
(RayTrainWorker pid=1456225) 2023-05-31 22:12:09,486	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)


...

(pid=1456225) - RandomizeBlockOrder 1:   0%|          | 0/200 [00:00<?, ?it/s]

(pid=1456225) Running 0:   0%|          | 0/200 [00:00<?, ?it/s]

(RayTrainWorker pid=1456225) 2023-05-31 22:12:11,193	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[RandomizeBlockOrder]
(RayTrainWorker pid=1456225) 2023-05-31 22:12:11,194	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)


(RayTrainWorker pid=1456225) 


2023-05-31 22:12:14,653	INFO tune.py:945 -- Total run time: 13.99 seconds (13.97 seconds for the tuning loop).
(raylet) [2023-05-31 22:12:14,711 E 1453491 1453600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-05-31_22-08-32_654304_1453193 is over 95% full, available space: 17756790784; capacity: 418112987136. Object creation will fail if spilling is required.
(raylet) [2023-05-31 22:12:24,725 E 1453491 1453600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-05-31_22-08-32_654304_1453193 is over 95% full, available space: 17756831744; capacity: 418112987136. Object creation will fail if spilling is required.
(raylet) [2023-05-31 22:12:34,736 E 1453491 1453600] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-05-31_22-08-32_654304_1453193 is over 95% full, available space: 17756798976; capacity: 418112987136. Object creation will fail if spilling is required.
(raylet) [2023-05-31 22:12:44,748 E 1453491 1453600] (raylet) file_system_monitor.cc:111: /tmp